In [19]:
# Import Libraries # 
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib as plt

# Load spreadsheet # 
timetrace = pd.read_excel('Anomaly Cleaning 1.xlsx')
time_interval = timetrace.shape[0]
print(time_interval)
# Size sheet
data = timetrace.to_numpy()
size = data.size
col = int(size/len(data))
row = len(data)
print(col)
print(row)
rowtrun = row - 11
print(data[1:5,0])

# Time converter
def convert_time(time_strings):
    time_format = "%m-%d-%Y %H:%M:%S"
    base_time = datetime.strptime(time_strings[0], time_format)
    
    time_in_minutes = [(datetime.strptime(t, time_format) - base_time).total_seconds() / 60 for t in time_strings]
    return time_in_minutes

# Steady state checker presently set to 20 points - Work in Progress!!
def steady_state(range1, threshold, timeint, threshold2, threshold3):
    g = len(range1) - 1
    steadystatecount = 0
    for i in range(g):
        p = i + 1
        grad = (range1[p] - range1[i])/ (timeint[p] - timeint[i])
        delta = range1[p] - range1[i]
        if abs(grad) > abs(threshold) or abs(delta) > abs(threshold2):
            steadystatecount = 0
            continue
        else:
            steadystatecount += 1
        
        if steadystatecount > 18:
            totval = 0
            steadystateval = 0
            r = i - 20
            for k in range(r,i):
                totval += range1[r]
                steadystateval = totval/20
            return [True, steadystateval]

2872
7
2872
['09-04-2023 10:54:42' '09-04-2023 10:54:45' '09-04-2023 10:54:48'
 '09-04-2023 10:54:51']


In [23]:
# Find anomalous windows
# Initialise arrays
anoms = []
foptdstarts = []
foptdends = []

# Find all possible anomalies: Change range to (1,col), j range to (31,rowtrun) when done, adjust values 30, 31, 5, etc based on steady state length
for i in range(2,3):
    sum = 0
    for j in range(31,rowtrun):
        q = j - 20

        # Detect non steady state points
        timerange = convert_time(data[q:j,0])
        c = data[q:j,i].tolist()
        state = steady_state(c, 100, timerange, 10, 100)
        if state == None:
            anoms.append([data[0,i],j,i,data[j,0]])

print(anoms[0])
timetrace2 = timetrace
for i in range(len(anoms)):
    anomrow = anoms[i][1] 
    timetrace2 = timetrace2.drop(timetrace.index[anomrow])

print(timetrace2)
timetrace2.to_excel(r'C:\\Users\\jonat\\Data.xlsx', index=False)

['FT109', 45, 2, '09-04-2023 10:56:54']
               Unnamed: 0  Unnamed: 1  Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0          Log Time Stamp       FT108       FT109      LT108     PT115a   
1     09-04-2023 10:54:42   36.873375   49.902122  78.575577   0.845031   
2     09-04-2023 10:54:45   62.339409    92.44825  78.491463   0.845071   
3     09-04-2023 10:54:48   74.283493   97.560913  78.381752   0.845097   
4     09-04-2023 10:54:51   76.812111   98.606979  78.243423   0.845105   
...                   ...         ...         ...        ...        ...   
2867  09-04-2023 16:31:12  168.747696  182.325134   49.29702   0.858024   
2868  09-04-2023 16:31:15  116.160439  124.606903  49.693413   0.858028   
2869  09-04-2023 16:31:18   85.759872  104.252602  50.096851   0.857989   
2870  09-04-2023 16:31:21   78.407364  100.957733  50.516903   0.857974   
2871  09-04-2023 16:31:24   76.563789  100.199409  50.906185   0.857933   

     Unnamed: 5    Unnamed: 6  
0        PT115b  Pump Speed